In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Shape creation functions
def create_scalene_triangle(center):
    x, y = center
    return np.array([
        [x, y],
        [x + 2, y + 0.5],
        [x + 0.5, y + 2]
    ])

def create_rectangle(center, width, height):
    x, y = center
    return np.array([
        [x - width/2, y - height/2],
        [x + width/2, y - height/2],
        [x + width/2, y + height/2],
        [x - width/2, y + height/2]
    ])

def create_secondary_triangle():
    start = np.array([0, 0])
    edge1 = np.array([3.0, 0])
    edge2_angle = np.radians(60)
    edge2 = np.array([2.06 * np.cos(edge2_angle), 2.06 * np.sin(edge2_angle)])
    point1 = start
    point2 = start + edge1
    point3 = point2 + edge2
    return np.array([point1, point2, point3])

# Geometry utilities
def rotate_shape(shape, angle_deg, origin=(0, 0)):
    angle = np.radians(angle_deg)
    rot_matrix = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])
    translated = shape - origin
    rotated = translated @ rot_matrix.T
    return rotated + origin

def annotate_edges(ax, shape, label_prefix):
    for i in range(len(shape)):
        start = shape[i]
        end = shape[(i + 1) % len(shape)]
        mid = (start + end) / 2
        ax.text(mid[0], mid[1], f"{label_prefix}{i}", fontsize=10, color='black', ha='center')

def edge_lengths(shape):
    return [np.linalg.norm(shape[i] - shape[(i + 1) % len(shape)]) for i in range(len(shape))]

def join_shapes(base, join, base_edge, join_edge):
    b_start = base[base_edge]
    b_end = base[(base_edge + 1) % len(base)]
    j_start = join[join_edge]
    j_end = join[(join_edge + 1) % len(join)]

    b_vec = b_end - b_start
    j_vec = j_end - j_start

    b_angle = np.arctan2(b_vec[1], b_vec[0])
    j_angle = np.arctan2(j_vec[1], j_vec[0])
    angle_diff = np.degrees(b_angle - j_angle + np.pi)

    rotated_join = rotate_shape(join, angle_diff, origin=j_start)
    j_rotated_start = rotated_join[join_edge]
    j_rotated_end = rotated_join[(join_edge + 1) % len(rotated_join)]

    translation = b_start - j_rotated_end
    translated_join = rotated_join + translation

    # Calculate edge lengths and determine the shorter one
    b_length = np.linalg.norm(b_end - b_start)
    j_length = np.linalg.norm(j_rotated_end - j_rotated_start)
    shorter_length = min(b_length, j_length)

    # Normalize vectors and scale to shorter length
    b_dir = b_vec / b_length
    seam_base = (b_start, b_start + b_dir * shorter_length)
    seam_join = (translated_join[(join_edge + 1) % len(translated_join)],
                 translated_join[join_edge])

    return translated_join, seam_base, seam_join

# Scraps
triangle = create_scalene_triangle(center=(3, 5))
rectangle = create_rectangle(center=(10, 5), width=3, height=1.2)
secondary_triangle = create_secondary_triangle() + np.array([6, 10])
scraps = {"T": triangle, "R": rectangle, "S": secondary_triangle}

# Seam log
seam_log = []

def show_join_three(scrap1, edge1, scrap2, edge2, scrap3, join_target, edge3, target_edge):
    shape1 = scraps[scrap1]
    shape2 = scraps[scrap2]
    shape2_transformed, seam_base1, seam_join1 = join_shapes(shape1, shape2, edge1, edge2)

    target_shape = shape1 if join_target == scrap1 else shape2_transformed
    shape3 = scraps[scrap3]
    shape3_transformed, seam_base2, seam_join2 = join_shapes(target_shape, shape3, target_edge, edge3)

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 30)
    ax.set_ylim(0, 30)
    ax.set_aspect('equal')
    ax.set_title(f"Join {scrap1}{edge1} to {scrap2}{edge2}, then {scrap3}{edge3} to {join_target}{target_edge}")

    colors = {'T': 'lightblue', 'R': 'salmon', 'S': 'lightgreen'}
    ax.fill(shape1[:, 0], shape1[:, 1], color=colors[scrap1], alpha=0.6, label=f"{scrap1}")
    ax.fill(shape2_transformed[:, 0], shape2_transformed[:, 1], color=colors[scrap2], alpha=0.6, label=f"{scrap2}")
    ax.fill(shape3_transformed[:, 0], shape3_transformed[:, 1], color=colors[scrap3], alpha=0.6, label=f"{scrap3}")

    annotate_edges(ax, shape1, scrap1)
    annotate_edges(ax, shape2_transformed, scrap2)
    annotate_edges(ax, shape3_transformed, scrap3)

    # Highlight shorter edge for seam between shape1 and shape2
    ax.plot([seam_base1[0][0], seam_base1[1][0]], [seam_base1[0][1], seam_base1[1][1]], color='purple', linewidth=3)
    # Highlight shorter edge for seam between target_shape and shape3
    ax.plot([seam_base2[0][0], seam_base2[1][0]], [seam_base2[0][1], seam_base2[1][1]], color='purple', linewidth=3)

    ax.legend()
    plt.grid(True)
    plt.show()

    seam_log.append({
        "scrap1": scrap1,
        "edge1": edge1,
        "scrap2": scrap2,
        "edge2": edge2,
        "scrap3": scrap3,
        "join_target": join_target,
        "edge3": edge3,
        "target_edge": target_edge
    })
    print("Seam log:", seam_log)

# Interactive interface
scrap_keys = list(scraps.keys())
widgets.interact(
    show_join_three,
    scrap1=widgets.Dropdown(options=scrap_keys, value='T', description='Scrap A'),
    edge1=widgets.IntSlider(min=0, max=3, step=1, value=0, description='Edge A'),
    scrap2=widgets.Dropdown(options=scrap_keys, value='R', description='Scrap B'),
    edge2=widgets.IntSlider(min=0, max=3, step=1, value=0, description='Edge B'),
    scrap3=widgets.Dropdown(options=scrap_keys, value='S', description='Scrap C'),
    join_target=widgets.Dropdown(options=['T', 'R'], value='R', description='Join C To'),
    edge3=widgets.IntSlider(min=0, max=3, step=1, value=0, description='Edge C'),
    target_edge=widgets.IntSlider(min=0, max=3, step=1, value=0, description='Edge Target')
)


interactive(children=(Dropdown(description='Scrap A', options=('T', 'R', 'S'), value='T'), IntSlider(value=0, …

<function __main__.show_join_three(scrap1, edge1, scrap2, edge2, scrap3, join_target, edge3, target_edge)>

In [ ]:
voila your_notebook.ipynb
